In [42]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing #カリフォルニア
import pandas as pd
import matplotlib.pyplot as plt

import nnabla as nn

import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
import random
from nnabla.utils.data_iterator import data_iterator_simple
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [32]:
california_housing = fetch_california_housing()

train_x = pd.DataFrame(california_housing.data, columns=california_housing.feature_names)
train_y = pd.Series(california_housing.target)

train_x.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [33]:
train_y.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
dtype: float64

In [54]:
dataset = fetch_california_housing()
Y = dataset.target
X = dataset.data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

1.768

In [43]:
scaler = StandardScaler()
X_n_train = scaler.fit_transform(X_train)
X_n_test = scaler.fit_transform(X_test)

データイテレーターを作成する

ロジスティック回帰

In [98]:
nn.clear_parameters()
batch_size = 1
x = nn.Variable([batch_size, X_n_train.shape[1]])
with nn.parameter_scope("affine1"):
    y = PF.affine(x, 1)

In [99]:
t = nn.Variable([batch_size, 1])
loss = F.mean(F.squared_error(y, t))

In [100]:
x.d = X_n_train[0]
t.d = Y_train[0]

loss.forward()

print("Prediction score of 0-th data: {}".format(y.d[0]))
print("Loss: {}".format(loss.d))

Prediction score of 0-th data: [0.2826733]
Loss: [[2.1648018]]


In [101]:
for param in nn.get_parameters().values():
    param.grad.zero()

In [95]:
learning_rate = 1e-3
solver = S.Sgd(learning_rate)
solver.set_parameters(nn.get_parameters())

In [102]:
for i in range(10000):
    x.d = X_n_train[i]
    t.d = Y_train[i]
    loss.forward()
    solver.zero_grad()
    loss.backward()
    solver.weight_decay(1e-5)
    solver.update()
    if i % 100 == 0:
        print(i, loss.d)

0 [[2.1648018]]
100 [[0.3816582]]
200 [[0.00048597]]
300 [[0.0906549]]
400 [[1.5977465]]
500 [[11.44877]]
600 [[18.012714]]
700 [[4.1168604]]
800 [[1.4857538]]
900 [[1.9136187]]
1000 [[6.7746367]]
1100 [[2.3688946]]
1200 [[0.03599136]]
1300 [[1.7128993]]
1400 [[1.8210493]]
1500 [[1.1657758]]
1600 [[3.5766783]]
1700 [[0.38501948]]
1800 [[14.40823]]
1900 [[6.677495]]
2000 [[1.2303945]]
2100 [[0.090253]]
2200 [[28.616821]]
2300 [[17.111729]]
2400 [[0.36629027]]
2500 [[7.1439776]]
2600 [[1.0822377]]
2700 [[19.23507]]
2800 [[3.1832201]]
2900 [[8.706331]]
3000 [[18.657137]]
3100 [[0.3839931]]
3200 [[0.04930806]]
3300 [[2.8348083]]
3400 [[16.87339]]
3500 [[4.93681]]
3600 [[28.732561]]
3700 [[19.93936]]
3800 [[0.23694594]]
3900 [[23.650938]]
4000 [[1.8091245]]
4100 [[1.9403822]]
4200 [[4.747394]]
4300 [[0.6054769]]
4400 [[0.10855418]]
4500 [[2.9895427]]
4600 [[1.8518913]]
4700 [[37.549408]]
4800 [[0.4009971]]
4900 [[7.4020047]]
5000 [[10.8351]]
5100 [[0.11285497]]
5200 [[17.195507]]
5300 [[0.4

In [103]:
x.d = X_n_test[0]
t.d = Y_test[0]
y.forward()
print(x.d)
print(t.d)
print(y.d)

[[ 0.48149693 -0.3747834   0.64401533 -0.25555544 -0.22315972 -0.02960821
  -0.30358683 -0.44299567]]
[[1.768]]
[[0.47090527]]


2層のニューラルネットワーク

In [6]:
nn.clear_parameters()
def Create_double_network():
    x = nn.Variable([None, X_train.shape[1]])
    with nn.parameter_scope("midle"):
        h = F.tanh(PF.affine(x, 32))
    with nn.parameter_scope("out"):
        y = PF.affine(h, 1)
    return x, y

#x, y = Create_double_network()
#print("Shapes:", x.shape, y.shape)

In [ ]:
nn.get_parameters()

OrderedDict()

In [ ]:
x.d = X_train
y.forward()
print(y.d)

NameError: name 'x' is not defined